In [ ]:
!pip install pyspark

**KHÔNG SỬ DỤNG SPARK**

In [ ]:
import csv
import numpy as np
import math

In [ ]:
def loadData(path):
    f = open(path, "r")
    data = csv.reader(f) #csv format
    data = np.array(list(data))# covert to matrix
    data = np.delete(data, 0, 0)# delete header
    data = np.delete(data, 0, 1) # delete index
    np.random.shuffle(data) # shuffle data
    f.close()
    trainSet = data[:] #training data from 1->100
    testSet = data[:]# the others is testing data
    return trainSet, testSet

In [ ]:
def calcDistancs(pointA, pointB, numOfFeature=4):
    tmp = 0
    for i in range(numOfFeature):
        tmp += (float(pointA[i]) - float(pointB[i])) ** 2
    return math.sqrt(tmp)

In [ ]:
def kNearestNeighbor(trainSet, point, k):
    distances = []
    for item in trainSet:
        distances.append({
            "label": item[-1],
            "value": calcDistancs(item, point,6)
        })
    distances.sort(key=lambda x: x["value"])
    labels = [item["label"] for item in distances]
    return labels[:k]

In [ ]:
def findMostOccur(arr):
    labels = set(arr) # set label
    ans = ""
    maxOccur = 0
    for label in labels:
        num = arr.count(label)
        if num > maxOccur:
            maxOccur = num
            ans = label
    return ans

In [ ]:
trainSet, testSet = loadData("abalone.csv")
start = timer()

for item in testSet:
    knn = kNearestNeighbor(trainSet, item, 3)
    answer = findMostOccur(knn)
    print("Số thứ tự trong dataset: {} -> predicted: {}".format(item[-1], answer))
end = timer()
print('Thời gian thực thi' ,end-start)

Số thứ tự trong dataset: 12 -> predicted: 10
Số thứ tự trong dataset: 9 -> predicted: 9
Số thứ tự trong dataset: 8 -> predicted: 8
Số thứ tự trong dataset: 21 -> predicted: 16
Số thứ tự trong dataset: 11 -> predicted: 11
Số thứ tự trong dataset: 11 -> predicted: 11
Số thứ tự trong dataset: 11 -> predicted: 9
Số thứ tự trong dataset: 11 -> predicted: 11
Số thứ tự trong dataset: 10 -> predicted: 10
Số thứ tự trong dataset: 6 -> predicted: 6
Số thứ tự trong dataset: 9 -> predicted: 9
Số thứ tự trong dataset: 12 -> predicted: 13
Số thứ tự trong dataset: 25 -> predicted: 13
Số thứ tự trong dataset: 8 -> predicted: 9
Số thứ tự trong dataset: 14 -> predicted: 7
Số thứ tự trong dataset: 13 -> predicted: 13
Số thứ tự trong dataset: 9 -> predicted: 9
Số thứ tự trong dataset: 8 -> predicted: 10
Số thứ tự trong dataset: 8 -> predicted: 8
Số thứ tự trong dataset: 9 -> predicted: 8
Số thứ tự trong dataset: 9 -> predicted: 10
Số thứ tự trong dataset: 9 -> predicted: 9
Số thứ tự trong dataset: 5 -> pr

**SỬ DỤNG SPARK**

In [ ]:
import pandas as pd
from pyspark.sql.functions import col, explode
from pyspark import SparkContext
from pyspark.sql import SparkSession
from timeit import default_timer as timer
from pyspark import SparkConf, SparkContext
import collections
import numpy as np
from itertools import islice, count
from sympy import isprime, nextprime

In [ ]:
conf = SparkConf().setMaster("local").setAppName("KNN")
sc = SparkContext.getOrCreate(conf=conf)

In [ ]:
trainSet, testSet = loadData("abalone.csv")

In [ ]:
rdd = sc.parallelize(testSet)

In [ ]:
def knn(item):
    knn = kNearestNeighbor(trainSet, item, 3)
    answer = findMostOccur(knn)
    return item[-1],answer

In [ ]:
 start= timer()
 kq = rdd.map(lambda x : knn(x))
 end = timer()
 print('Thời gian thực thi' ,end-start)

Thời gian thực thi 0.0060803739997936646
